In [ ]:
!uv pip install transformers datasets
!uv pip install sentencepiece
!uv pip install 'accelerate>=0.26.0'

Using Python 3.12.11 environment at: /home/zeus/miniconda3/envs/cloudspace
Audited 2 packages in 9ms
Using Python 3.12.11 environment at: /home/zeus/miniconda3/envs/cloudspace
Resolved 39 packages in 269ms                                        
Prepared 1 package in 68ms                                               
░░░░░░░░░░░░░░░░░░░░ [0/1] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 1 package in 25ms                                 
 + accelerate==1.12.0


In [ ]:
# !unzip  "Language Challenge.zip"

In [1]:
import numpy as np 
import pandas as pd
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                         TrainingArguments, DataCollatorWithPadding)
from datasets import Dataset
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score, precision_score, recall_score
import torch
from transformers import Trainer
from torch import nn
from datasets import ClassLabel

In [2]:
train = pd.read_csv('Language Challenge/quora-question-pairs/train.csv')
test = pd.read_csv('Language Challenge/quora-question-pairs/test.csv')

train["question1"] = train["question1"].astype(str)
train["question2"] = train["question2"].astype(str)

test["question1"] = test["question1"].astype(str)
test["question2"] = test["question2"].astype(str)

/tmp/ipykernel_29873/3094735980.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('Language Challenge/quora-question-pairs/test.csv')


In [4]:
class_weights = compute_class_weight(
    "balanced",
    classes=np.array([0, 1]),
    y=train["is_duplicate"].values
)

class_weights = torch.tensor(class_weights, dtype=torch.float)

print(f"Class weights: {class_weights}")  

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

def tokenize_function(examples):
    return tokenizer(
        examples["question1"],
        examples["question2"],
        truncation=True,
        padding=False,
        max_length=350
    )

# Convert to HF dataset
dataset = Dataset.from_pandas(train[["question1", "question2", "is_duplicate"]])
dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.rename_column("is_duplicate", "labels")
dataset = dataset.cast_column("labels", ClassLabel(names=['non_duplicate', 'duplicate']))

split_dataset = dataset.train_test_split(
    test_size=0.1, 
    stratify_by_column="labels",
    seed=42
)

Class weights: tensor([0.7926, 1.3543])


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/404290 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/404290 [00:00<?, ? examples/s]

In [ ]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False,num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                       labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss
    

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/deberta-v3-base",
    num_labels=2
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./deberta-tuned-task2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=2500,
    save_strategy="steps",
    save_steps=2500,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_steps=100,
    warmup_steps=500, 
    fp16=True

)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'f1': f1_score(labels, predictions, average='binary'),
        'f1_macro': f1_score(labels, predictions, average='macro'),
        'precision': precision_score(labels, predictions),
        'recall': recall_score(labels, predictions),
    }

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.class_weights = class_weights.to(trainer.args.device)

trainer.train()


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_29873/1255542706.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Step,Training Loss,Validation Loss,F1,F1 Macro,Precision,Recall
2500,0.252800,0.270286,0.858170,0.879910,0.781291,0.951829
5000,0.232200,0.221934,0.880331,0.901203,0.827497,0.940373
7500,0.167000,0.229015,0.885843,0.906327,0.839695,0.937358
10000,0.172500,0.221047,0.889842,0.910265,0.852288,0.930859


TrainOutput(global_step=11372, training_loss=0.22155164582101217, metrics={'train_runtime': 4383.5639, 'train_samples_per_second': 166.011, 'train_steps_per_second': 2.594, 'total_flos': 2.165844790637431e+16, 'train_loss': 0.22155164582101217, 'epoch': 2.0})

In [ ]:

MODEL_DIR = "deberta-task2"   

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)  
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)  

device = "cuda" if torch.cuda.is_available() else "cpu"

model = model.to(device)
model = torch.compile(model)
model.eval()


The tokenizer you are loading from 'deberta-tuned-task2/checkpoint-11372' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


OptimizedModule(
  (_orig_mod): DebertaV2ForSequenceClassification(
    (deberta): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128100, 768, padding_idx=0)
        (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-11): 12 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=768, out_features=768, bias=True)
                (key_proj): Linear(in_features=768, out_features=768, bias=True)
                (value_proj): Linear(in_features=768, out_features=768, bias=True)
                (pos_dropout): Dropout(p=0.1, inplace=False)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): DebertaV2SelfOutput(
                (dense): Linear(in_featu

In [ ]:
test_df = train  

test_ds = Dataset.from_pandas(test_df[["question1", "question2"]])

def tok_fn(batch):
    return tokenizer(
        batch["question1"],
        batch["question2"],
        truncation=True,          
        max_length=350,           
    )

test_ds = test_ds.map(
    tok_fn,
    batched=True,
    remove_columns=["question1", "question2"],  
)

collator = DataCollatorWithPadding(tokenizer=tokenizer)
loader = DataLoader(test_ds, batch_size=256, shuffle=False, collate_fn=collator)

all_logits = []
with torch.no_grad():
    for batch in tqdm(loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(**batch).logits
        all_logits.append(logits.cpu())

logits = torch.cat(all_logits, dim=0).numpy()
probs = torch.softmax(torch.tensor(logits), dim=1).numpy()

pred_label = np.argmax(logits, axis=1)  
p_duplicate = probs[:, 1]               

test_df["pred_label"] = pred_label
test_df["p_duplicate"] = p_duplicate


Map:   0%|          | 0/404290 [00:00<?, ? examples/s]

  0%|          | 0/1580 [00:00<?, ?it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/_inductor/compile_fx.py:282: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0112 09:25:09.385000 29873 /system/conda/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode
100%|██████████| 1580/1580 [22:43<00:00,  1.16it/s]


In [10]:
submission = test_df[["id", "pred_label"]].copy()
submission.columns = ["id", "prediction"]
submission.to_csv("task2_pred.csv", index=False)

In [ ]:
y_true = train["is_duplicate"].values
y_pred = train["pred_label"].values

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)

Precision: 0.8906495184763545
Recall: 0.9696642838479731
F1: 0.9284788689025173


In [ ]:
y_true = train["is_duplicate"].values
probs = train["p_duplicate"].values

best_f1 = 0
best_t = 0
best_precision = 0
best_recall = 0
for t in np.linspace(0.05, 0.95, 50):
    f1 = f1_score(y_true, probs > t)
    precision = precision_score(y_true, probs > t)
    recall = recall_score(y_true, probs > t)
    if f1 > best_f1:
        best_f1 = f1
        best_precision = precision
        best_recall = recall

print("Best threshold:", best_t)
print("Best F1:", best_f1)
print("Best Precision:", best_precision)
print("Best Recall:", best_recall)

Best threshold: 0
Best F1: 0.9307837363513659
Best Precision: 0.9086396120469628
Best Recall: 0.9540341544790069
